# update_prices.ipynb
This notebook guides through the action of updating the prices of a database, i.e. adding records in the `price` table with the prices registered on the current day.

The notebook is divided in two parts:
1. In the first part we create a minimal database with a single book. We save it as `database.db` in the current folder.
2. In the second part we load the database `database.db` and we read it as a `polars.DataFrame`. We then read all the rows of the database and scrape the prices for any link we find. The prices are then added to the table `prices` in the database with the current date.

In [1]:
from datetime import datetime

import monitorbookprices as mbp

We load the database `database.db` in the current folder, which is read as a `polars.DataFrame` with a specific schema (specified by `mbp.schema()`)

In [2]:
books = mbp.read_database(
    'books',  # table name
    url='sqlite:///database.db',
    schema_overrides=mbp.schema(),
)
books

isbn,author,title,year,publisher,full_price,adelphi,buecher,feltrinelli,ibs,libraccio,mondadori,osiander
str,str,str,str,str,f64,str,str,str,str,str,str,str
"""9783866473256""","""Karl Marx""","""Das Kapital""","""2009""","""Anaconda""",7.95,null,"""https://www.buecher.de/artikel…",null,null,null,null,"""https://www.osiander.de/shop/h…"
"""9781857988826""","""Ursula K. Le Guin""","""The Dispossessed""","""1999""","""Orion Publishing Co""",13.0,null,"""https://www.buecher.de/artikel…","""https://www.lafeltrinelli.it/d…","""https://www.ibs.it/dispossesse…",null,null,"""https://www.osiander.de/shop/h…"


As a first step, we define empty lists, we store today's date, and the book's isbn.

In [3]:
list_isbns = []
list_sites = []
list_prices = []
list_dates = []
date = datetime.today().date()

Notice that we need to keep in mind that for each book we do not have links for all webistes, for example we only have the following websites for the first book:

In [4]:
[s.name for s in books[0][mbp.list_sites()] if not s.null_count()]

['buecher', 'osiander']

Then, the following function extracts the following lists:
| `list_isbn` |    `list_sites`     |
|-------------|---------------------|
|    isbn_1   |  link_1 for book_1  |
|    isbn_1   |  link_2 for book_1  |
|    isbn_1   |  link_3 for book_1  |
|    isbn_2   |  link_1 for book_2  |
|    isbn_2   |  link_2 for book_2  |

etc.

In [5]:
# list_isbn, list_site = mbp.prepare_scrape(books)
# list_isbn, list_site

In [6]:
###----------------- ITERATIVE -------------------
# %%time
# if row.shape[0] > 0:
#     for p in row:
#         list_isbns.append(isbn)
#         list_sites.append(p.name)
#         list_prices.append(mbp.scrape_price(p[0]))
#         list_dates.append(date)

In [7]:
# %%time
# ###----------------------- PARALLEL -----------------
# from concurrent.futures import ThreadPoolExecutor

# # def run(func, lista):
# #     with ThreadPoolExecutor(max_workers=6) as executor:
# #         results = list(tqdm(executor.map(func, lista), total=len(lista)))
# #     return results

# # list_prices = run(mbp.scrape_price, [p[0] for p in row])

# with ThreadPoolExecutor(max_workers=6) as executor:
#     list_price = list(
#         tqdm(
#             executor.map(
#                 mbp.scrape_price,
#                 list_site
#             ),
#         total=len(list_site)
#         )
#     )

# list_dates = [date]*len(list_prices)

In [8]:
# pl.DataFrame(
#     {
#         'isbn': list_isbns,
#         'site': list_sites,
#         'price': list_prices,
#         'date': list_dates,
#     }
# )

In [9]:
prices = mbp.scrape_database(books)
prices

100%|██████████| 6/6 [00:24<00:00,  4.01s/it]


isbn,site,price,date
str,str,f64,date
"""9783866473256""","""https://www.buecher.de/artikel…",7.95,2024-10-10
"""9783866473256""","""https://www.osiander.de/shop/h…",7.95,2024-10-10
"""9781857988826""","""https://www.buecher.de/artikel…",9.49,2024-10-10
"""9781857988826""","""https://www.lafeltrinelli.it/d…",12.49,2024-10-10
"""9781857988826""","""https://www.ibs.it/dispossesse…",11.87,2024-10-10
"""9781857988826""","""https://www.osiander.de/shop/h…",9.69,2024-10-10


After scraping the prices, we can add these records to the `prices` table in the database with
```python
mbp.write_database(
    prices,
    table_name='prices',
    url='sqlite:///database.db',
    if_table_exists='append'
)
```